In [34]:

import warnings
from utils import collect_error_metrics
import numpy as np
import pandas as pd
import xgboost
from hyperopt import fmin, hp, space_eval, tpe, STATUS_OK, Trials, partial
from hyperopt.pyll import scope
from sklearn import set_config
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.svm import SVR
from plotly import express as px
from plotly import graph_objects as go
from plotly import offline as pyo
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import cross_val_score, KFold
from sklearn.utils import check_random_state

warnings.filterwarnings('ignore')

set_config(display='diagram')


In [35]:
df = pd.read_parquet('./Data/Tifton_SPI_FE.parquet')
df = df.drop(['date', 'hour'], axis=1)

In [36]:
# def add_lags(df):
#     target_map = df['SPI'].to_dict()
#     df['lag1'] = (df.index - pd.Timedelta('364 days')).map(target_map)
#     df['lag2'] = (df.index - pd.Timedelta('728 days')).map(target_map)
#     df['lag3'] = (df.index - pd.Timedelta('1092 days')).map(target_map)
#     return df

In [37]:
# df = add_lags(df)

In [38]:
le = LabelEncoder()
df['season'] = le.fit_transform(df['season'])
df['weekday'] = le.fit_transform(df['weekday'])
df['day'] = le.fit_transform(df['day'])

In [39]:
train = df.loc[df.index < '01-01-2015']
test = df.loc[df.index >= '01-01-2015']

In [42]:
FEATURES = ['prcp_accum', 'air_temp_avg', 'smp_2', 'smp_4', 'smp_8', 'smp_20',
            'smp_40', 'soil_temp_2', 'soil_temp_4', 'soil_temp_8', 'soil_temp_20',
            'soil_temp_40', 'wind_dir_avg', 'wind_speed_avg', 'PRCP', 'year',
            'month', 'day', 'dayofweek', 'weekday', 'quarter', 'dayofyear',
            'dayofmonth', 'weekofyear', 'date_offset', 'season', 'soil_temp_avg',
            'smp_avg']
TARGET = 'SPI'

In [43]:
X_train = train[FEATURES]
y_train = train[TARGET]
X_test = test[FEATURES]
y_test = test[TARGET]

In [44]:

numeric_features = FEATURES[0:15] + FEATURES[26:28]
numeric_transformer = Pipeline(
    steps=[('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())])
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features)
    ],
    remainder='passthrough',
)
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['prcp_accum', 'air_temp_avg', 'smp_2',
                                  'smp_4', 'smp_8', 'smp_20', 'smp_40',
                                  'soil_temp_2', 'soil_temp_4', 'soil_temp_8',
                                  'soil_temp_20', 'soil_temp_40',
                                  'wind_dir_avg', 'wind_speed_avg', 'PRCP'])])

In [45]:
import joblib
joblib.dump(preprocessor, 'preprocessor.pkl')

['preprocessor.pkl']

In [46]:
GRADIENT_BOOSTING_REGRESSOR = "gradient_boosting_regressor"
KWARGS = "kwargs"
LEARNING_RATE = "learning_rate"
LINEAR_REGRESSION = "linear_regression"
MAX_DEPTH = "max_depth"
MODEL = "model"
MODEL_CHOICE = "model_choice"
NORMALIZE = "normalize"
N_ESTIMATORS = "n_estimators"
RANDOM_FOREST_REGRESSOR = "random_forest_regressor"
RANDOM_STATE = "random_state"
SGD_REGRESSOR = "stochastic_gradient_descent_regressor"
XGB_REGRESSOR = "xgboost_regressor"
SEED = 0

stochastic_gradient_descent_regressor = {
    MODEL: SGD_REGRESSOR,
    KWARGS: {
        'loss': hp.choice(f'{SGD_REGRESSOR}__loss',
                          ['squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive']),
        'penalty': hp.choice(f'{SGD_REGRESSOR}__penalty', ['none', 'l2', 'l1', 'elasticnet']),
        'alpha': hp.uniform(f'{SGD_REGRESSOR}__alpha', 0.0001, 0.1),
        'l1_ratio': hp.uniform(f'{SGD_REGRESSOR}__l1_ratio', 0, 1),
        'max_iter': hp.choice(f'{SGD_REGRESSOR}__max_iter', [100, 200, 300, 400, 500]),
        'epsilon': hp.uniform(f'{SGD_REGRESSOR}__epsilon', 0.001, 0.1),
        'learning_rate': hp.choice(f'{SGD_REGRESSOR}__learning_rate',
                                   ['constant', 'optimal', 'invscaling', 'adaptive']),
        'eta0': hp.uniform(f'{SGD_REGRESSOR}__eta0', 0.0001, 1),
        'power_t': hp.uniform(f'{SGD_REGRESSOR}__power_t', 0.5, 1.5),
        'validation_fraction': hp.uniform(f'{SGD_REGRESSOR}__validation_fraction', 0.1, 0.5),
        'n_iter_no_change': hp.choice(f'{SGD_REGRESSOR}__n_iter_no_change', [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]),
        RANDOM_STATE: 0,
        'early_stopping': True,
        'verbose': 3,
        'tol': 0.1
    },
}

xtreme_gradient_boost_regressor = {
    MODEL: XGB_REGRESSOR,
    KWARGS: {

        'max_depth': hp.choice(f'{XGB_REGRESSOR}__max_depth', [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]),
        'gamma': hp.uniform(f'{XGB_REGRESSOR}__gamma', 1, 9),
        'reg_alpha': hp.quniform(f'{XGB_REGRESSOR}__reg_alpha', 40, 180, 1),
        'reg_lambda': hp.uniform(f'{XGB_REGRESSOR}__reg_lambda', 0, 1),
        'colsample_bytree': hp.uniform(f'{XGB_REGRESSOR}__colsample_bytree', 0.5, 1),
        'min_child_weight': hp.quniform(f'{XGB_REGRESSOR}__min_child_weight', 0, 10, 1),
        'n_estimators': hp.choice(f'{XGB_REGRESSOR}__n_estimators', [80, 100, 200, 300, 400, 500]),
        'learning_rate': hp.choice(f'{XGB_REGRESSOR}__learning_rate', [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]),
        'seed': 0,

    }
}

random_forest_regressor = {
    MODEL: RANDOM_FOREST_REGRESSOR,
    KWARGS: {
        N_ESTIMATORS: scope.int(
            hp.quniform(f"{RANDOM_FOREST_REGRESSOR}__{N_ESTIMATORS}", 50, 150, 1)
        ),
        MAX_DEPTH: scope.int(
            hp.quniform(f"{RANDOM_FOREST_REGRESSOR}__{MAX_DEPTH}", 2, 12, 1)
        ),
        RANDOM_STATE: 0,
    },
}

gradient_boosting_regressor = {
    MODEL: GRADIENT_BOOSTING_REGRESSOR,
    KWARGS: {
        LEARNING_RATE: scope.float(
            hp.uniform(f"{GRADIENT_BOOSTING_REGRESSOR}__{LEARNING_RATE}", 0.01, 0.15, )
        ),
        N_ESTIMATORS: scope.int(
            hp.quniform(f"{GRADIENT_BOOSTING_REGRESSOR}__{N_ESTIMATORS}", 50, 150, 1)
        ),
        MAX_DEPTH: scope.int(
            hp.quniform(f"{GRADIENT_BOOSTING_REGRESSOR}__{MAX_DEPTH}", 2, 12, 1)
        ),
        RANDOM_STATE: 0,
    },
}


In [47]:
space = {
    MODEL_CHOICE: hp.choice(
        MODEL_CHOICE, [random_forest_regressor, gradient_boosting_regressor, stochastic_gradient_descent_regressor,
                       xtreme_gradient_boost_regressor],
    )
}

In [48]:
LOSS = 'loss'
STATUS = 'status'

MODELS = {
    GRADIENT_BOOSTING_REGRESSOR: GradientBoostingRegressor,
    RANDOM_FOREST_REGRESSOR: RandomForestRegressor,
    SGD_REGRESSOR: SGDRegressor,
    XGB_REGRESSOR: xgboost.XGBRegressor
}

mse_scorer = make_scorer(mean_squared_error)


def sample_to_model(sample):
    kwargs = sample[MODEL_CHOICE][KWARGS]
    return MODELS[sample[MODEL_CHOICE][MODEL]](**kwargs)


def objective(sample, dataset_df, features, target):
    model = sample_to_model(sample)
    rng = check_random_state(0)
    cv = KFold(n_splits=10, random_state=rng, shuffle=True)
    mse = cross_val_score(
        model,
        preprocessor.fit_transform(dataset_df.loc[:, features]),
        dataset_df.loc[:, target],
        scoring=mse_scorer,
        cv=cv,
        n_jobs=-1, error_score=0.99
    )
    return {LOSS: np.mean(mse), STATUS: STATUS_OK}

In [49]:
SPI_objective = partial(
    objective, dataset_df=df, features=FEATURES, target=TARGET
)

In [50]:
trials = Trials()

best = fmin(SPI_objective, space, tpe.suggest, 1000, trials=trials, rstate=np.random.default_rng(SEED))

 36%|███▌      | 358/1000 [04:07<07:24,  1.44trial/s, best loss: 3656.1965566377417]


KeyboardInterrupt: 

In [ ]:
best_params = space_eval(space, best)

In [ ]:
best

In [ ]:
best = {k.replace('gradient_boosting_regressor__',''): v for k, v in best.items()}
best['max_depth'] = int(best['max_depth'])
best['n_estimators'] = int(best['n_estimators'])
best

In [ ]:
X_train = train[FEATURES]
y_train = train[TARGET]
X_test = test[FEATURES]
y_test = test[TARGET]

####################################
GBR_pipe = Pipeline(steps=[('preprocessing', preprocessor), ('reg', GradientBoostingRegressor(**best))])
GBR_pipe

In [ ]:
import joblib
joblib.dump(GBR_pipe, 'HyperParamTuned_GRB_Pipeline_Model.pkl')

In [ ]:

################################
GBR_pipe.fit(X_train, y_train)
y_pred = GBR_pipe.predict(X_test)
test['SPI_PRED'] = y_pred
#####################################
RMSE, MAE, MAPE = collect_error_metrics(test, 'SPI', 'SPI_PRED')
score = r2_score(y_test, y_pred)
print(RMSE,MAE,MAPE)
print("The accuracy of our model is {}%".format(round(score, 2) *100))


# model_stats.loc[len(model_stats.index)] = [model_names_list[est], RMSE, MAE, MAPE, score]

In [ ]:
spi_all = pd.concat([train,test], sort = False)
_ = spi_all[['SPI','SPI_PRED']].plot(figsize=(15, 5))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
_ = spi_all[['SPI_PRED','SPI']].plot(ax=ax,
                                              style=['-','.'])
ax.set_ylim(-3, 3)
ax.set_xbound(lower='01-01-2016', upper='02-01-2016')
plot = plt.suptitle('Month of predictions, Jan, 2016')

In [ ]:
def unpack(x):
    if x:
        return x[0]
    return np.nan


def handleNaming(x):
    if x == 0:
        return RANDOM_FOREST_REGRESSOR
    elif x == 1:
        return GRADIENT_BOOSTING_REGRESSOR
    elif x == 2:
        return SGD_REGRESSOR
    else:
        return XGB_REGRESSOR

In [ ]:
# def extract_top_n_models(trials, n):
#     """
#     Extract top n^th models and their parameters from a HyperOpt Trials object
#
#     Parameters
#     ----------
#     trials : HyperOpt Trials object
#         HyperOpt Trials object
#     n : int
#         Number of top models to extract
#
#     Returns
#     -------
#     top_n_models : list
#         List of top n^th models
#     top_n_params : list
#         List of top n^th models' parameters
#     """
#     top_n_models = []
#     top_n_params = []
#     for i in range(n):
#         top_n_models.append(trials.best_trial['result']['model'])
#         top_n_params.append(trials.best_trial['misc']['vals'])
#     return top_n_models, top_n_params
#
# top_n_models, top_n_params = extract_top_n_models(trials, 10)
#


In [ ]:
# We'll first turn each trial into a series and then stack those series together as a dataframe.
trials_df = pd.DataFrame([pd.Series(t["misc"]["vals"]).apply(unpack) for t in trials])
# Then we'll add other relevant bits of information to the correct rows and perform a couple of
# mappings for convenience
trials_df["loss"] = [t["result"]["loss"] for t in trials]
trials_df["trial_number"] = trials_df.index
trials_df[MODEL_CHOICE] = trials_df[MODEL_CHOICE].apply(
    lambda x: RANDOM_FOREST_REGRESSOR if x == 0 else GRADIENT_BOOSTING_REGRESSOR
)

In [ ]:


# We'll first turn each trial into a series and then stack those series together as a dataframe.
trials_df = pd.DataFrame([pd.Series(t["misc"]["vals"]).apply(unpack) for t in trials])
# Then we'll add other relevant bits of information to the correct rows and perform a couple of
# mappings for convenience
trials_df["loss"] = [t["result"]["loss"] for t in trials]
trials_df["trial_number"] = trials_df.index
trials_df[MODEL_CHOICE] = trials_df[MODEL_CHOICE].apply(
    lambda x: handleNaming(x)
)

In [ ]:
from plotly import express as px
from plotly import graph_objects as go
from plotly import offline as pyo

pyo.init_notebook_mode()

In [ ]:
def extract_top_models_from_hyperopt(trials_df):
    best_params_per_model = trials_df.groupby(['model_choice']).min().sort_values(by='loss', ascending=True)
    MultiModelParams = []
    for i in range(len(list(best_params_per_model.index))):
        n_params_series = best_params_per_model.iloc[i].dropna().drop(['loss', 'trial_number'])
        n_model_name = n_params_series.name
        temp = {i: {'model_name': n_model_name}}
        repl = f"{n_model_name}__"
        param_dict = {}
        for j in n_params_series.index:
            param_dict[j.replace(repl, "")] = n_params_series[j]
            try:
                if param_dict['max_depth'] is not None:
                    param_dict['max_depth'] = int(param_dict['max_depth'])
                if param_dict['n_estimators'] is not None:
                    param_dict['n_estimators'] = int(param_dict['n_estimators'])

            except KeyError:
                continue
        if  temp[i]['model_name'] == 'stochastic_gradient_descent_regressor':
                param_dict['n_iter_no_change'] = 15
                param_dict['max_iter'] = 200
                param_dict['penalty'] = 'l2'
        temp[i]['params'] = param_dict
        MultiModelParams.append(temp)
    return MultiModelParams


In [ ]:
top_models_params = extract_top_models_from_hyperopt(trials_df)

In [ ]:
top_models_params

In [ ]:

model_names_to_class = {
    'gradient_boosting_regressor': GradientBoostingRegressor,
    'random_forest_regressor': RandomForestRegressor,
    'xgboost_regressor': xgboost.XGBRegressor,
    'stochastic_gradient_descent_regressor': SGDRegressor
}


In [ ]:

from utils import collect_error_metrics

def generate_multi_model_lists(model_params, model_names_to_class):
    my_model_params = model_params
    estimator_list = []
    model_names_list = []
    for i in range(len(my_model_params)):
        model_name = my_model_params[i][i]['model_name']
        model = model_names_to_class[model_name]
        untuned_n_pipe = model()
        params = my_model_params[i][i]['params']
        tuned_n_pipe = model(**params)
        ###############################
        estimator_list.append(untuned_n_pipe)
        model_names_list.append(f"untuned_{model_name}")
        ################################
        estimator_list.append(tuned_n_pipe)
        model_names_list.append(model_name)
    return estimator_list, model_names_list


estimator_list, model_names_list = generate_multi_model_lists(model_params=top_models_params,
                                                              model_names_to_class=model_names_to_class)


In [ ]:
estimator_list

In [ ]:

def run_multi_model(train, test, FEATURES, TARGET, estimator_list, model_names_list):
    pred_data = {}
    model_stats = pd.DataFrame({'model_name': [], 'real_mean_squared_error': [], 'mean_absolute_error': [],
                                'mean_absolute_percentage_error': [], 'accuracy_score': []})
    n_classes = 3
    CLFS = []
    y_scores = []
    for est in range(len(estimator_list)):
        n_test = test.copy()
        X_train = train[FEATURES]
        y_train = train[TARGET]
        X_test = n_test[FEATURES]
        y_test = n_test[TARGET]
        X = preprocessor.fit_transform(X_train.loc[:, FEATURES])
        ################################
        clf = estimator_list[est]
        clf.fit(X=X, y=y_train)
        CLFS.append(clf)
        y_pred = clf.predict(X_test)
        n_test['SPI_PRED'] = y_pred
        pred_data[model_names_list[est]] = y_pred
        #####################################
        RMSE, MAE, MAPE = collect_error_metrics(n_test, 'SPI', 'SPI_PRED')
        score = r2_score(y_test, y_pred)
        model_stats.loc[len(model_stats.index)] = [model_names_list[est], RMSE, MAE, MAPE, score]

    return pred_data, CLFS, model_stats


pred_data, CLFS, model_stats = run_multi_model(train=train, test=test, FEATURES=FEATURES, TARGET=TARGET, estimator_list=estimator_list, model_names_list=model_names_list)


In [ ]:

[i.replace(list(best_params_per_model.iloc[0].dropna().drop(['loss', 'trial_number']).index)

 n_model_params = []





In [ ]:
import seaborn as sns

sns.scatterplot(data=trials_df, x='trial_number', y='loss', hue=MODEL_CHOICE)